In [1]:
import os
import pandas as pd
import great_expectations as gx
from great_expectations.data_context import FileDataContext

In [2]:
import streamlit as st
import numpy as np
from io import StringIO

In [3]:
path_to_repo_dir="/Users/shruti/Desktop/Assignment_1/Part_2/GEX_Origination" #local path to csv
path_to_data_dir=f"{path_to_repo_dir}/gx/data" 
expectation_suite_name="GE_Suite" 

In [4]:
context = FileDataContext.create(project_root_dir=path_to_repo_dir)

In [5]:
os.mkdir(path_to_data_dir)

df = pd.read_csv('/Users/shruti/Desktop/Assignment_1/Part_2/datasets/sample_orig_2021.txt', delimiter="|", header = None)



In [6]:
df.columns=['Credit_Score', 'First_Payment_Date', 'First_Time_Homebuyer_Flag','Maturity_Date','MSA', 'MI','Number_of_Units','Occupancy_Status','CLTV','DTI','UPB','LTV','Original_Interest_Rate','Channel','PPM','Amortization','Property_State','Property_Type','Postal_Code','Loan_Sequence_Number','Loan_Purpose','Original_Loan_Term','Number_of_Borrowers','Seller_Name','Servicer_Name','Super_Conforming_Flag','HARP','Program_Indicator','HARP_Indicator','Property_Valuation_Method','nterest_Only','Mortgage_Insurance_Cancellation_Indicator']

# Save the DataFrame as a CSV file
df.to_csv(f"{path_to_repo_dir}/gx/data/historical_data_2022.csv", index=False)

In [7]:
df.head()

,Credit_Score,First_Payment_Date,First_Time_Homebuyer_Flag,Maturity_Date,MSA,MI,Number_of_Units,Occupancy_Status,CLTV,DTI,...,Number_of_Borrowers,Seller_Name,Servicer_Name,Super_Conforming_Flag,HARP,Program_Indicator,HARP_Indicator,Property_Valuation_Method,nterest_Only,Mortgage_Insurance_Cancellation_Indicator
0,776,202103,N,205102,NaN,0,1,P,52,12,...,1,Other sellers,Other servicers,NaN,NaN,9,NaN,3,N,7
1,746,202103,N,205102,NaN,30,1,P,95,33,...,1,Other sellers,Other servicers,NaN,NaN,9,NaN,2,N,N
2,710,202103,N,204102,NaN,0,1,P,65,50,...,1,Other sellers,Other servicers,NaN,NaN,9,NaN,2,N,7
3,722,202103,N,205102,16984.0,25,1,P,89,40,...,2,Other sellers,Other servicers,NaN,NaN,9,NaN,1,N,N
4,761,202103,N,205102,29020.0,0,1,P,56,50,...,1,Other sellers,Other servicers,NaN,NaN,9,NaN,2,N,7


In [8]:
# Give your Datasource a name
datasource_name = "Local_FileSystem_Source"
datasource = context.sources.add_pandas_filesystem(name=datasource_name, base_directory=path_to_data_dir)

# Give your first Asset a name
asset_name = "Freddie_Mac"
batching_regex = r"historical_data_\d{4}.csv" #change this

asset = datasource.add_csv_asset(name=asset_name,batching_regex=batching_regex)

# Build batch request
batch_request = asset.build_batch_request()

Create expectations suite

In [9]:
data_asset = context.get_datasource(datasource_name).get_asset(asset_name)
batch_request = data_asset.build_batch_request()

In [10]:
context.list_expectation_suite_names()

[]

In [11]:
context. add_or_update_expectation_suite(expectation_suite_name)

{
  "expectation_suite_name": "GE_Suite",
  "ge_cloud_id": null,
  "expectations": [],
  "data_asset_type": null,
  "meta": {
    "great_expectations_version": "0.17.19"
  }
}

In [12]:
validator = context.get_validator(batch_request=batch_request, expectation_suite_name=expectation_suite_name)
validator.head()

Calculating Metrics:   0%|          | 0/1 [00:00<?, ?it/s]

,Credit_Score,First_Payment_Date,First_Time_Homebuyer_Flag,Maturity_Date,MSA,MI,Number_of_Units,Occupancy_Status,CLTV,DTI,...,Number_of_Borrowers,Seller_Name,Servicer_Name,Super_Conforming_Flag,HARP,Program_Indicator,HARP_Indicator,Property_Valuation_Method,nterest_Only,Mortgage_Insurance_Cancellation_Indicator
0,776,202103,N,205102,NaN,0,1,P,52,12,...,1,Other sellers,Other servicers,NaN,NaN,9,NaN,3,N,7
1,746,202103,N,205102,NaN,30,1,P,95,33,...,1,Other sellers,Other servicers,NaN,NaN,9,NaN,2,N,N
2,710,202103,N,204102,NaN,0,1,P,65,50,...,1,Other sellers,Other servicers,NaN,NaN,9,NaN,2,N,7
3,722,202103,N,205102,16984.0,25,1,P,89,40,...,2,Other sellers,Other servicers,NaN,NaN,9,NaN,1,N,N
4,761,202103,N,205102,29020.0,0,1,P,56,50,...,1,Other sellers,Other servicers,NaN,NaN,9,NaN,2,N,7


Validators for schema

In [13]:
def validate_ge_origination():

    #schema check

    validator.expect_column_values_to_be_of_type("First_Payment_Date", "datetime")
    validator.expect_column_values_to_be_of_type("First_Time_Homebuyer_Flag", "str")
    validator.expect_column_values_to_be_of_type("Maturity_Date", "datetime")
    validator.expect_column_values_to_be_of_type("MSA", "int")
    validator.expect_column_values_to_be_of_type("MI", "int")
    validator.expect_column_values_to_be_of_type("Number_of_Units", "int")
    validator.expect_column_values_to_be_of_type("Occupancy_Status", "str")
    validator.expect_column_values_to_be_of_type("CLTV", "float")
    validator.expect_column_values_to_be_of_type("DTI", "float")
    validator.expect_column_values_to_be_of_type("UPB", "int")
    validator.expect_column_values_to_be_of_type("LTV", "float")
    validator.expect_column_values_to_be_of_type("Original_Interest_Rate", "float")

    validator.expect_column_values_to_be_of_type("Channel", "str")

    validator.expect_column_values_to_be_of_type("PPM", "str")

    validator.expect_column_values_to_be_of_type("Amortization", "str")

    validator.expect_column_values_to_be_of_type("Property_State", "str")

    validator.expect_column_values_to_be_of_type("Property_Type", "str")

    validator.expect_column_values_to_be_of_type("Postal_Code", "int")

    validator.expect_column_values_to_be_of_type("Loan_Sequence_Number", "str")

    validator.expect_column_values_to_be_of_type("Loan_Purpose", "str")

    validator.expect_column_values_to_be_of_type("Original_Loan_Term", "int")

    validator.expect_column_values_to_be_of_type("Number_of_Borrowers", "int")

    validator.expect_column_values_to_be_of_type("Seller_Name", "str")

    validator.expect_column_values_to_be_of_type("Servicer_Name", "str")

    validator.expect_column_values_to_be_of_type("Super_Conforming_Flag", "str")

    validator.expect_column_values_to_be_of_type("HARP", "str")

    validator.expect_column_values_to_be_of_type("Program_Indicator", "str")

    validator.expect_column_values_to_be_of_type("HARP_Indicator", "str")

    validator.expect_column_values_to_be_of_type("Property_Valuation_Method", "int")

    validator.expect_column_values_to_be_of_type("nterest_Only", "str")

    validator.expect_column_values_to_be_of_type("Mortgage_Insurance_Cancellation_Indicator", "str")

 

 

    #Validation

    validator.expect_column_values_to_be_in_set ("First_Time_Homebuyer_Flag", ["Y","N"])

    validator.expect_column_values_to_be_in_set("Occupancy_Status", ["P", "I", "S"])

    validator.expect_column_values_to_be_in_set("Channel", ["B", "R", "C"])

    validator.expect_column_values_to_match_regex("Loan_Sequence_Number", regex=r'^P\d{2}Qn[a-zA-Z0-9]{7}$')

    validator.expect_column_values_to_match_regex("HARP_Indicator", regex=r'^P\d{2}Qn[a-zA-Z0-9]{7}$')

    validator.expect_column_value_lengths_to_be_between("Credit_Score", min_value=1, max_value=4)

    validator.expect_column_value_lengths_to_be_between("UPB", min_value=1, max_value=12)

    validator.expect_column_values_to_be_in_set("Amortization", ["FRM", "ARM"])

    validator.expect_column_values_to_match_regex("Postal_Code", r"^\d{5}$")

    validator.expect_column_values_to_match_regex("Original_Interest_Rate", regex=r'^\d{3}\.\d{3}$')

    expected_states = ["AL", "AK", "AZ", "AR", "CA", "CO", "CT", "DE", "FL", "GA", "HI", "ID", "IL", "IN", "IA", "KS", "KY", "LA", "ME", "MD", "MA", "MI", "MN", "MS", "MO", "MT", "NE", "NV", "NH", "NJ", "NM", "NY", "NC", "ND", "OH", "OK", "OR", "PA", "RI", "SC", "SD", "TN", "TX", "UT", "VT", "VA", "WA", "WV", "WI", "WY"]

    validator.expect_column_values_to_be_in_set("Property_State", value_set=expected_states)

    

    #Missing data

    for column in df.columns:

        validator.expect_column_values_to_not_be_null(column)

 

 

    validator.save_expectation_suite()

 



In [14]:
validate_ge_origination()

Calculating Metrics:   0%|          | 0/1 [00:00<?, ?it/s]

Calculating Metrics:   0%|          | 0/8 [00:00<?, ?it/s]

Calculating Metrics:   0%|          | 0/1 [00:00<?, ?it/s]

Calculating Metrics:   0%|          | 0/1 [00:00<?, ?it/s]

Calculating Metrics:   0%|          | 0/1 [00:00<?, ?it/s]

...

Calculating Metrics:   0%|          | 0/6 [00:00<?, ?it/s]

Calculating Metrics:   0%|          | 0/6 [00:00<?, ?it/s]

Calculating Metrics:   0%|          | 0/6 [00:00<?, ?it/s]

Calculating Metrics:   0%|          | 0/6 [00:00<?, ?it/s]

Calculating Metrics:   0%|          | 0/6 [00:00<?, ?it/s]

In [15]:
checkpoint = context.add_or_update_checkpoint(
    name="Freddie_Mac_Origination_v1",
    validator=validator
)

In [16]:
checkpoint_result = checkpoint.run(run_name="Manual_run")

Calculating Metrics:   0%|          | 0/259 [00:00<?, ?it/s]

In [17]:
context.build_data_docs()

{'local_site': 'file:///Users/shruti/Desktop/Assignment_1/Part_2/GEX_Origination/gx/uncommitted/data_docs/local_site/index.html'}